In [41]:
import os
import json
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.llms import HuggingFaceHub
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from huggingface_hub import login

from langchain.prompts import PromptTemplate


In [42]:
load_dotenv()
hf_api_key = os.getenv('HUGGINGFACEHUB_API_TOKEN')
login(token=hf_api_key)

In [43]:
with open('candidates.json', 'r') as f:
    data = json.load(f)

In [44]:
candidate_docs = []
for emp in data['employees']:
    doc = (
        f"{emp['name']} has {emp['experience_years']} years of experience. "
        f"Skills: {', '.join(emp['skills'])}. "
        f"Projects: {', '.join(emp['projects'])}. "
        f"Availability: {emp['availability']}."
    )
    candidate_docs.append(doc)

In [45]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
documents = text_splitter.create_documents(candidate_docs)

In [46]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vector_db = FAISS.from_documents(documents, embeddings)
retriever = vector_db.as_retriever(search_kwargs={"k": 2})


In [47]:
llm = CTransformers(model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_type="mistral")

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 15.89it/s]


In [ ]:
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an HR assistant. Based on the following context about available candidates, answer the user's request with a warm, structured, human-like recommendation listing the top candidates with their details.

Context:
{context}

User Request:
{question}

Answer in this format:
"Based on your requirements for <requirements>, I found an excellent candidates:
1) <Name>...
2) <Name>...
Both have the technical depth and domain expertise you need. Would you like me to provide more details about their specific projects or check their availability for meetings?"
"""
)

In [49]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": custom_prompt}
)


In [50]:
def ask_hr_bot(query):
    response = qa_chain.run(query)
    return response

In [53]:
print(ask_hr_bot('Find physics teacher with 3+ years experience'))


Answer:
Based on your requirements for a physics teacher with 3+ years of experience, I found X excellent candidates:
1) Vivaan Gupta - He has 2 years of experience and skills in React Native, JavaScript, UI/UX. He's worked on Fitness App and Online Learning App projects. He's available now.
2) Divya Shetty - She also has 2 years of experience and skills in React, JavaScript, UI/UX. She's worked on Job Portal Frontend project. She's available now.
Both have the technical depth and domain expertise you need. Would you like me to provide more details about their specific projects or check their availability for meetings?
